![embedding_mapping.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/embedding_mapping.png)

## Homework: Un(semi)-supervised word translation learning

Homework based on [Conneau et al. 2018](https://arxiv.org/abs/1710.04087) article.

In the homework we offer you to train a mapping between Ukrainian word vectors and Russian word vectors just like in the first homework of the NLP course. But unlike the first homework this mapping will be build (almost) unsupervised: without parallel data (pairs of corresponding words in Ukrainian and Russian).

In [ ]:
%env KERAS_BACKEND=tensorflow
%env CUDA_VISIBLE_DEVICES=1
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import keras
from keras.models import Sequential
from keras import layers as L
import numpy as np
import gensim

from IPython import display
from tqdm import tnrange
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
!wget https://www.dropbox.com/s/cnwyfbfa44mqxph/ukr_rus.train.txt?dl=1 -O ./ukr_rus.train.txt
!wget https://www.dropbox.com/s/78otz1d4d9b0284/ukr_rus.test.txt?dl=1 -O ./ukr_rus.test.txt
!wget https://www.dropbox.com/s/210m7gwqkikpsxd/uk.w2v.bin?dl=1 -O ./uk.w2v.bin
!wget https://www.dropbox.com/s/3luwyjdmofsdfjz/ru.w2v.bin?dl=1 -O ./ru.w2v.bin

In [ ]:
ru_embs = gensim.models.KeyedVectors.load_word2vec_format("ru.w2v.bin", binary=True)
uk_embs = gensim.models.KeyedVectors.load_word2vec_format("uk.w2v.bin", binary=True)

In [ ]:
x = uk_embs.vectors[:50000]
y = ru_embs.vectors[:50000]

In [ ]:
def precision(pairs, uk_vectors, topn=1):
    """ TODO maybe insert docstring """
    assert len(pairs) == len(uk_vectors)
    num_matches = 0
    for i, (uk, ru) in enumerate(pairs):
        num_matches += ru in set(w[0] for w in ru_embs.most_similar([uk_vectors[i]], topn=topn))
    return num_matches / len(pairs)

def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r") as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_embs or ru not in ru_embs:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_embs[uk])
            ru_vectors.append(ru_embs[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [ ]:
uk_ru_test, x_test, y_test = load_word_pairs("ukr_rus.test.txt")
uk_ru_train, x_train, y_train = load_word_pairs("ukr_rus.train.txt")

In [ ]:
precision(uk_ru_test, x_test, 5)

## Reminder

### Embedding space mapping

Let $x_i \in \mathrm{R}^d$ be the distributed representation of word $i$ in the source language, and $y_i \in \mathrm{R}^d$ is the vector representation of its translation. Our purpose is to learn such linear transform $W$ that minimizes euclidian distance between $Wx_i$ and $y_i$ for some subset of word embeddings. Thus we can formulate so-called Procrustes problem:

$$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$$
or
$$W^*= \arg\min_W ||WX - Y||_F$$

where $||*||_F$ - Frobenius norm.

In Greek mythology, Procrustes or "the stretcher" was a rogue smith and bandit from Attica who attacked people by stretching them or cutting off their legs, so as to force them to fit the size of an iron bed. We make same bad things with source embedding space. Our Procrustean bed is target embedding space.

But wait...$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$ looks like simple multiple linear regression (without intercept fit). So let's code.

### Orthogonal Procrustean Problem

It can be shown (see original paper) that a self-consistent linear mapping between semantic spaces should be orthogonal. TODO simplify phrases
We can restrict transform $W$ to be orthogonal. Then we will solve next problem:

$$W^*= \arg\min_W ||WX - Y||_F \text{, where: } W^TW = I$$

$$I \text{- identity matrix}$$

Instead of making yet another regression problem we can find optimal orthogonal transformation using singular value decomposition. It turns out that optimal transformation $W^*$ can be expressed via SVD components:
$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$
$$W^*=UV^T$$

## Word translation learning using GAN (8 points)

### Generator

If $\mathcal{X}=\{x_1,...,x_n\} \subset \mathrm{R}^d$ - source embedding set, and $\mathcal{Y}=\{y_1,...,y_m\} \subset \mathrm{R}^d$ - target embedding set, then discriminator is simply orthogonal mapping that can be defined as square matrix: $W\in O_d(\mathrm{R})$.

In terms of neural network, generator is a network with single linear layer with orthogonality constraint and without nonlinearity after it.

The generator input is a source embedding $x_i$, the generator output is a mapped source embedding $Wx_i$

In [ ]:
EMB_SIZE = 300

In [ ]:
import keras, keras.layers as L

def build_generator(emb_size):
    # TIPS: use keras.Sequential and keras.initializers
    # YOUR_CODE
    return model

In [ ]:
generator = build_generator(EMB_SIZE)

### Discriminator

Discriminator is a neural network that should discriminate between objects from $W\mathcal{X}$ (mapped source embeddings) and objects from $\mathcal{Y}$ (target embeddings).

Just like in original article for discriminator we will use a multilayer perceptron with two hidden layers of size 2048, and Leaky-ReLU activation functions. The input to the discriminator is corrupted with dropout noise
with a rate of 0.1.

The discriminator input is either mapped source embedding $Wx_i$ or target embedding $y_j$, the discriminator output is a probability of input to be from source distribution $p_D=p_D(source=1)$



In [ ]:
def build_discriminator(emb_size):
    # YOUR_CODE
    return model

In [ ]:
discriminator = build_discriminator(EMB_SIZE)

### Discriminator loss

The purpose of discriminator is to maximize output probability for mapped source embeddings $p_D(source=1|Wx_i)$ and minimize probability for target embeddings $p_D(source=1|y_j)$. The last is equivalent to maximization of  $p_D(source=0|y_j)$. Thus, we can train this classifier with standard cross-entropy loss:

$$\mathcal{L}_D(\theta_D|W)=-\frac{1}{n}\sum_{i=1}^n\log p_D(source=1|Wx_i)-\frac{1}{m}\sum_{i=1}^m\log p_D(source=0|y_i)$$
Equivalent:
$$\mathcal{L}_D(\theta_D|W)=-\frac{1}{n}\sum_{i=1}^n\log p_D(source=1|Wx_i)-\frac{1}{m}\sum_{i=1}^m\log (1-p_D(source=1|y_i))$$

**NB:** We minimize $\mathcal{L}_D(\theta_D|W)$ with respect discriminator parameters $\theta_D$. The matrix $W$ is fixed.

In [ ]:
# YOUR_CODE HERE

#X = 
#Y = 
#W = 
#WX =

#logp_wx_is_real = 
#logp_wx_is_fake = 
#logp_y_is_real = 

# L_d = L_d_source + L_d_target

As suggested Goodfellow (2016) it is useful to use soft targets instead hard ones. In case label smoothing:
$$\mathcal{L}_D(\theta_D|W)=\mathcal{L}_{D_1}+\mathcal{L}_{D_2}$$

Where:
$$\mathcal{L}_{D_1}=\frac{1}{n}\sum_{i=1}^n[(1-\alpha)\log p_D(source=1|Wx_i) + \alpha\log p_D(source=0|Wx_i)]$$

$$\mathcal{L}_{D_2}=\frac{1}{m}\sum_{i=1}^n[(1-\alpha)\log p_D(source=0|Wx_i) + \alpha\log p_D(source=1|Wx_i)]$$

In [ ]:
# YOUR CODE HERE IF YOU REALLY WANT TO USE LABEL SMOOTHING

### Generator loss

The purpose of generator is to fool discriminator, i.e. to produce mapping $W\mathcal{X}$ indistinguishable from $\mathcal{Y}$. Therefore we should turn over discriminator loss, minimize output probability for mapped source embeddings $p_D(source=1|Wx_i)$ and minimize probability for target embeddings $p_D(source=0|y_j)$.

$$\mathcal{L}_G(W|\theta_D)=-\frac{1}{n}\sum_{i=1}^n\log (1-p_D(source=1|Wx_i))-\frac{1}{m}\sum_{i=1}^m\log p_D(source=1|y_i)$$

**NB:** We minimize $\mathcal{L}_G(W|\theta_D)$ with respect matrix $W$ coefficients. Disciminator parameters $\theta_D$ is fixed.

Because gradients do not flow through generator for target samples:

$$\mathcal{L}_G(W|\theta_D)=-\frac{1}{n}\sum_{i=1}^n\log (1-p_D(source=1|Wx_i))$$

In contrast with original article to be more stable we allow you to add a supervised component of loss - MSE for small number of fixed pairs vectors from $\mathcal{X}$ and $\mathcal{Y}$.

$$\mathcal{L}_G(W|\theta_D)=-\frac{1}{n}\sum_{i=1}^n\log (1-p_D(source=1|Wx_i))+\gamma \frac{1}{N}\sum_{k}^N(Wx_k-y_k)^2$$

In [ ]:
#X_pair = tf.placeholder('float32', [None, EMB_SIZE])
#Y_pair = tf.placeholder('float32', [None, EMB_SIZE])

# YOUR_CODE_HERE

#L_g = L_mse * 100 + L_g_source

### Orthogonality constraint
Conneau et al. propose to use a simple update step to ensure that the matrix $W$ stays close to an
orthogonal matrix during training:

$$W \gets (1+\beta)W-\beta(WW^T)W$$

In [ ]:
BETA = tf.constant(0.01)

# TIPS: USE tf.assing


### Training

In [ ]:
# LEARNING_RATE = 0.1
# GRADIENT DESCENT OPTIMIZER?

#gen_optim =
#dis_optim =

In [ ]:
BATCH_SIZE = 32

def sample_batch(bsize):
    x_batch = x[np.random.choice(np.arange(x.shape[0]), size=bsize)]
    y_batch = y[np.random.choice(np.arange(y.shape[0]), size=bsize)]
    return x_batch, y_batch

In [ ]:
def discriminator_step():
    # YOUR_CODE
    
def generator_step():
    # YOUR_CODE

def orthogonolize_step():
    sess.run(orthogonolize)


In [ ]:
def get_metrics():
    feed_dict = {
        X: x_test,
        Y: y_test,
        X_pair: x_train[:50],
        Y_pair: y_train[:50]
    }
    loss_g, loss_d, logp_x, logp_y, wx = sess.run([L_g, L_d, logp_wx_is_real, logp_y_is_real, WX], feed_dict)
    return loss_g, loss_d, np.exp(logp_x), np.exp(logp_y), wx

In [ ]:
sess = keras.backend.get_session()
sess.run(tf.global_variables_initializer())

In [ ]:
N_EPOCHS = 10
EPOCH_SIZE = 1000
DIS_STEPS = 5
GEN_STEPS = 1


gen_loss_history = []
dis_loss_history = []
prec_history = []

for epoch_num in range(N_EPOCHS):
    print("Epoch: {}".format(epoch_num + 1))
    for batch_num in tnrange(EPOCH_SIZE):
        
        for _ in range(DIS_STEPS):
            # YOUR_CODE
            
        for _ in range(GEN_STEPS):
            # YOUR_CODE
        
        if batch_num % 10 == 0:
            display.clear_output(wait=True)
            loss_g, loss_d, p_x, p_y, wx = get_metrics()
            gen_loss_history.append(loss_g)
            dis_loss_history.append(loss_d)
            
            if batch_num % 100 == 0:
                prec_history.append(precision(uk_ru_test, wx, 5))

            plt.figure(figsize=(15,15))
            plt.subplot(212)
            plt.plot(gen_loss_history, label="Generator loss")
            plt.plot(dis_loss_history, label="Discriminator loss")
            plt.legend(loc='best')

            plt.subplot(221)
            plt.title('Mapped vs target data')
            plt.hist(p_x, label='D(Y)', alpha=0.5,range=[0,1], bins=20)
            plt.hist(p_y, label='D(WX)',alpha=0.5,range=[0,1], bins=20)
            plt.legend(loc='best')
            
            plt.subplot(222)
            plt.title('Precision top5')
            plt.plot(prec_history)
            plt.show()
                
        

## Fully unsupervised word translation learning (2 points)

Try to exclude MSE term from generator loss and train GAN with sufficient quality (~40% precision top5). You should tune parameters of optimizers and training schedule to make it stable.

Good luck!